In [1]:
!nvidia-smi

Mon Feb 24 01:51:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti     Off |   00000000:01:00.0  On |                  N/A |
| N/A   55C    P8              3W /   50W |      55MiB /   4096MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!pipx install ultralytics
!pipx install ipywidgets
!pipx install pandas --include-deps
!pipx install matplotlib --include-deps
!pipx install opencv-python --include-deps
!pipx install lxml --include-deps
!pipx install scikit-learn --include-deps

'ultralytics' already seems to be installed. Not modifying existing
installation in '/home/vihanga-ms/.local/share/pipx/venvs/ultralytics'. Pass
'--force' to force installation.
'ipywidgets' already seems to be installed. Not modifying existing
installation in '/home/vihanga-ms/.local/share/pipx/venvs/ipywidgets'. Pass
'--force' to force installation.
  installed package pandas 2.2.3, installed using Python 3.12.3
  These apps are now globally available
    - f2py
    - numpy-config
done! ✨ 🌟 ✨
⚠️  File exists at /home/vihanga-ms/.local/bin/f2py and points to
    /home/vihanga-ms/.local/share/pipx/venvs/pandas/bin/f2py, not
    /home/vihanga-ms/.local/share/pipx/venvs/matplotlib/bin/f2py. Not
    modifying.
⚠️  File exists at /home/vihanga-ms/.local/bin/numpy-config and points to
    /home/vihanga-ms/.local/share/pipx/venvs/pandas/bin/numpy-config, not
    /home/vihanga-ms/.local/share/pipx/venvs/matplotlib/bin/numpy-config. Not
    modifying.
  installed package matplotlib 3.10.0, ins

In [10]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

ModuleNotFoundError: No module named 'ultralytics'

In [11]:
!pipx install tqdm --include-deps

'tqdm' already seems to be installed. Not modifying existing installation in
'/home/vihanga-ms/.local/share/pipx/venvs/tqdm'. Pass '--force' to force
installation.


In [12]:
import os
import random
import glob
import shutil

import json
import yaml

from collections import defaultdict
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

from lxml import etree as ET

import pandas as pd
import matplotlib.pyplot as plt

import cv2
# from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from IPython.display import display, Image

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
dataset_dir = "/kaggle/input/logodet3k/LogoDet-3K"
# dataset_dir = "/home/sinewave/logocode_detection/datasets/LogoDet-3K"
os.listdir(dataset_dir)

In [ ]:
df = pd.DataFrame(glob.glob(f"{dataset_dir}/*/*/*"), columns=["file_path"])
df["ext"] = df["file_path"].apply(lambda x: x.split(".")[-1])
df["logo_category"] = df["file_path"].apply(lambda x: x.split(os.sep)[-3])
df["logo_name"] = df["file_path"].apply(lambda x: x.split(os.sep)[-2])
df.head()

In [ ]:
# !du /kaggle/input/logodet3k/LogoDet-3K --human-readable -d 1
# !du /home/sinewave/logocode_detection/datasets/LogoDet-3K --human-readable -d 1

In [ ]:
df.to_csv("logodet3k_reference.csv", index=False)

In [ ]:
df["ext"].value_counts()

In [ ]:
df2 = df[df["ext"]=="jpg"].iloc[:]
print({
    "No. of categories": df2["logo_category"].nunique(), 
    "No. of logo types": df2["logo_name"].nunique(),
    "Avg. no. of images per category": df2.groupby("logo_category")["file_path"].count().mean().round(), 
    "Avg. no. of images per logo": df2.groupby("logo_name")["file_path"].count().mean().round(), 
})

In [ ]:
df2["logo_category"].value_counts()

In [ ]:
df2["logo_name"].value_counts().reset_index().plot(
    x="logo_name", y="count", figsize=(10,5), title="Distribution of logo img counts")

In [ ]:
dataset_dst_dir = "/kaggle/working/logodet3k"
if os.path.exists(dataset_dst_dir):
    shutil.rmtree(dataset_dst_dir)
os.makedirs(f"{dataset_dst_dir}/train", exist_ok=True)
os.makedirs(f"{dataset_dst_dir}/val", exist_ok=True)

classname2idx = {logo_name: idx for idx, logo_name in enumerate(sorted(df2["logo_name"].unique()))}
print(str(classname2idx)[:100]+"...")
idx2classname = {idx: logo_name for logo_name, idx in classname2idx.items()}

classname2idx = {"logo": 0}
idx2classname = defaultdict(lambda: "logo")

class_name_idx_map_str = "\n".join([f"    {idx}: {class_name}" for class_name, idx in classname2idx.items()])
print(class_name_idx_map_str)

dataset_config = f"""
path: {dataset_dst_dir} 
train:
    - train
val:
    - val

# test:
#     - test

# Classes
names:
{class_name_idx_map_str}
"""
print(dataset_config)
with open("dataset_config.yaml", "w") as f:
    f.write(dataset_config)
print("-"*10)
with open("dataset_config.yaml", "r") as f:
    datcon = yaml.safe_load(f)
    print(datcon)

In [ ]:
def convert_voc_to_yolo(src, dst, classname2idx):
    tree = ET.parse(src)
    root = tree.getroot()
    yolo_lines = []
    image_width = float(root.find("size/width").text)
    image_height = float(root.find("size/height").text)
    depth = float(root.find("size/depth").text)
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height
        class_index = classname2idx.get(class_name, 0)
#         yolo_line = f"{class_index} {round(x_center, 6)} {round(y_center, 6)} {round(width, 6)} {round(height, 6)}"
        yolo_line = f"{class_index} {x_center} {y_center} {width} {height}"
        yolo_lines.append(yolo_line)
    if dst is not None:
        with open(dst, "w") as f:
            f.write("\n".join(yolo_lines))
    return yolo_lines
    
convert_voc_to_yolo(f"{dataset_dir}/Clothes/2xist/1.xml", None, {})

In [ ]:
df2['is_train'] = True
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=101)
test_df['is_train'] = False
final_df = pd.concat([train_df, test_df])
final_df.reset_index(drop=True, inplace=True)

def copy_to_working(x):
    train_folder = "train" if x["is_train"] else "val"
    src = x["file_path"]
    dst = os.path.join(dataset_dst_dir, train_folder, "__".join(x["file_path"].split(os.sep)[-3:]))
    if not os.path.exists(dst):
        shutil.copy(src, dst)
    if not os.path.exists(dst.replace(".jpg", ".txt")):
        convert_voc_to_yolo(src.replace(".jpg", ".xml"), dst.replace(".jpg", ".txt"), classname2idx)
    return True

# copy_to_working(final_df.iloc[0].T.to_dict())
copy_to_working_results = []

with ThreadPoolExecutor() as e:
    for _, row in tqdm(final_df.iterrows()):
        status = e.submit(copy_to_working, dict(row))
        copy_to_working_results.append(status)
        
copy_to_working_results = final_df.apply(lambda x: copy_to_working(x), axis=1)
copy_to_working_results.sum(), final_df.shape[0]

In [ ]:
copy_to_working_results.sum(), final_df.shape[0]

In [ ]:
model = YOLO('yolov8m.pt')
# model = YOLO('yolov8x.pt')

In [ ]:
def plot_bboxes(r):
    annotator = Annotator(r.orig_img)
    boxes = r.boxes
    for box in boxes:
        b = box.xyxy[0]
        c = box.cls
        annotator.box_label(b, model.names[int(c)])
    img = annotator.result() 
    return img

In [ ]:
!wget -O person.jpg https://www.indiafilings.com/learn/wp-content/uploads/2023/03/Can-a-single-person-own-a-firm-in-India.jpg

In [ ]:
plt.imshow(plot_bboxes(model("person.jpg")[0]))
plt.show()

In [ ]:
logos_sample = list(df2.sample(10).T.to_dict().values())
results = model([logo["file_path"]for logo in logos_sample])
print(len(results))
for r in results:
    img = plot_bboxes(r)
    plt.imshow(img)
    plt.show()

In [ ]:
# !yolo train model=yolov8n.pt data=dataset_config.yaml epochs=3 imgsz=640
# !yolo export model=yolov8n_simple_train.pt
# !yolo val model=yolov8n_simple_train.pt data=dataset_config.yaml

In [ ]:
# img_path = random.choice(logos_sample)["file_path"]
# print("\n".join(convert_voc_to_yolo(img_path.replace(".jpg", ".xml"), None, {})))
# Image(img_path)

In [ ]:
test_results = model.train(data='dataset_config.yaml', epochs=10)
#, cache=True) # dont turn cache true it will run out of memory
print(test_results)

In [ ]:
val_results = model.val()
print(val_results)

In [ ]:
exported_model = model.export()

In [ ]:
print(exported_model)

In [ ]:
import os
for folder_name, _, files in os.walk("/kaggle/working/runs"):
    print(folder_name, files)

In [ ]:
# !tar czf /kaggle/working/runs.tar.gz /kaggle/working/runs

In [ ]:
# !ls 

In [ ]:
# model = YOLO(exported_model.replace(".torchscript",".pt"))
model = YOLO("/kaggle/input/yolo-single-logo/pytorch/v1/1/best_yolov8x_one_class.pt")

In [ ]:
results = model([logo["file_path"]for logo in logos_sample])
print(len(results))
for r in results:
    img = plot_bboxes(r)
    plt.imshow(img)
    plt.show()

In [ ]:
results = model(["https://s3-ap-southeast-1.amazonaws.com/assets.limetray.com/assets/user_images/logos/original/1602742091_DUNKINLogo.png",
                 "https://media.designrush.com/inspiration_images/134805/conversions/_1512076803_93_Nike-desktop.jpg"])
print(len(results))
for r in results:
    img = plot_bboxes(r)
    plt.imshow(img)
    plt.show()

In [ ]:
model